<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-09 13:01:27
-------------------
qualified stocks: 92
with latest results: 30
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.01 L
Current:  1.35 C
-------------------
Today PnL: 1.65 L (1.24%)
Current PnL: -14.74 L (-10.27%)
CY Booked + Current PnL: -8.66 L (-6.03%)
-------------------
Total profit:  3.95 L
Total loss:  -18.69 L
-------------------
Total Booked + Current PnL: 18.32 L (15.72%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.51%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.58 L (53.77%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LICI,785.05,983.0,-2.96,H-LC,83.70,238725.0,44033.0,5061.0,0.29,22.62,2.12,25.21,34.0,8.70,1.78,32.01,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,58.15,104469.0,26107.0,7919.0,-0.02,33.32,7.58,43.43,77.0,3.30,0.78,39.73,XY24,NTT,MISC
84,UNIONBANK,123.87,163.0,1.23,M-LC,91.30,177304.0,36464.0,8032.0,1.74,25.89,4.53,31.59,88.0,4.54,1.32,54.01,XY24,NTT,BANKS
61,RELIANCE,1291.83,1526.0,6.50,X-LC,35.87,158028.0,17219.0,8312.0,0.44,12.23,5.26,18.13,19.0,2.07,1.18,24.78,XY25,NTT,REFINERIES
77,TATAELXSI,7332.28,7332.0,-11.95,X-MC,67.93,92505.0,-10147.0,10139.0,2.08,-9.88,10.96,-0.00,55.0,-1.00,0.69,39.05,OX40N,NTT,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,54.89,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,260.0,-0.56,0.47,55.40,XR,NTT,HOTELS
47,JPPOWER,18.73,26.20,-34.40,L-SC,95.65,133512.0,-8330.0,64900.0,8.56,-5.87,48.61,39.88,256.0,-0.13,1.00,35.30,XY24,NTT,POWER
11,BANDHANBNK,214.25,400.00,-27.77,H-SC,80.43,215482.0,-33048.0,248515.0,7.04,-13.30,115.33,86.70,129.0,-0.13,1.61,42.55,XY24,NTT,BANKS
59,RBLBANK,230.61,388.00,49.48,M-SC,72.83,232664.0,-1636.0,161539.0,5.22,-0.70,69.43,68.25,242.0,-0.01,1.74,53.22,XY24,NTT,BANKS
29,HAPPSTMNDS,919.80,1491.84,-22.75,M-SC,44.57,65026.0,-31553.0,91615.0,5.16,-32.67,140.89,62.19,206.0,-0.34,0.49,14.02,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,561.33,680.00,-15.50,H-MC,42.39,231559.0,4220.0,43834.0,-1.68,1.86,18.93,21.14,118.0,0.10,1.73,29.65,XY24,NTT,PAINTS
17,CAMPUS,294.86,393.00,NaN,NaN,85.87,168481.0,706.0,55144.0,-0.92,0.42,32.73,33.28,200.0,0.01,1.26,30.44,XY24,NTT,FOOTWEAR
39,ICICIPRULI,600.83,791.05,-11.82,H-MC,47.83,142425.0,6637.0,36347.0,-0.92,4.89,25.52,31.66,119.0,0.18,1.06,17.44,X40,ATH,INSURANCE
54,MEDANTA,1064.65,1486.00,4.32,X-MC,23.37,233298.0,27821.0,53495.0,-0.60,13.54,22.93,39.58,67.0,0.52,1.74,25.02,XY24,NTT,HEALTHCARE
64,SAIL,130.64,228.00,103.46,M-MC,86.96,231230.0,6268.0,161375.0,-0.56,2.79,69.79,74.53,183.0,0.04,1.73,34.25,XY24,BTT,STEEL


In [12]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,49.36,M-SC,77.17,129789.0,-12690.0,13316.0,0.50,-8.91,10.26,0.44,241.0,-0.95,0.97,53.19,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-11.95,X-MC,67.93,92505.0,-10147.0,10139.0,2.08,-9.88,10.96,-0.00,55.0,-1.00,0.69,39.05,OX40N,NTT,IT
75,SYMPHONY,1306.42,1306.0,-27.85,M-SC,2.17,153860.0,-17281.0,17232.0,2.08,-10.10,11.20,-0.03,189.0,-1.00,1.15,10.58,OX40N,NTT,DURABLES
48,KANSAINER,299.63,340.0,-67.67,H-SC,6.52,225135.0,-44532.0,80868.0,0.40,-16.51,35.92,13.47,143.0,-0.55,1.68,12.69,XY24,NTT,PAINTS
82,TTKPRESTIG,769.29,770.0,-10.66,X-SC,16.30,81711.0,-19066.0,19161.0,0.39,-18.92,23.45,0.09,65.0,-1.00,0.61,6.33,OX40N,NTT,DURABLES


In [13]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-17.76,X-LC,11.96,267524.0,-28410.0,122312.0,1.35,-9.60,45.72,31.73,1.0,-0.23,2.00,7.08,X40,BTT,IT
43,INFY,1461.46,2275.00,-12.06,X-LC,40.22,282158.0,20557.0,125081.0,0.80,7.86,44.33,55.67,2.0,0.16,2.11,14.44,X40,BTT,IT
52,LTIM,5564.16,7197.33,-1.33,X-LC,69.57,189936.0,-10374.0,69175.0,1.38,-5.18,36.42,29.35,3.0,-0.15,1.42,31.22,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.26,X-LC,8.70,152687.0,-48699.0,136319.0,0.01,-24.18,89.28,43.51,5.0,-0.36,1.14,5.31,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-14.69,X-LC,20.65,268240.0,-2107.0,36749.0,0.22,-0.78,13.70,12.81,6.0,-0.06,2.00,11.02,XY25,NTT,FMCG


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UNIONBANK,123.87,163.0,1.23,M-LC,91.30,177304.0,36464.0,8032.0,1.74,25.89,4.53,31.59,88.0,4.54,1.32,54.01,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,58.15,104469.0,26107.0,7919.0,-0.02,33.32,7.58,43.43,77.0,3.30,0.78,39.73,XY24,NTT,MISC
51,LICI,785.05,983.0,-2.96,H-LC,83.70,238725.0,44033.0,5061.0,0.29,22.62,2.12,25.21,34.0,8.70,1.78,32.01,XY25,NTT,INSURANCE


In [15]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SAIL,130.64,228.00,103.46,M-MC,86.96,231230.0,6268.0,161375.0,-0.56,2.79,69.79,74.53,183.0,0.04,1.73,34.25,XY24,BTT,STEEL
36,HINDZINC,514.80,744.74,39.88,M-LC,93.48,104510.0,1550.0,44438.0,4.04,1.51,42.52,44.67,68.0,0.03,0.78,34.87,X5K,ATH,METALS
18,CAMS,3643.00,5250.99,7.11,H-SC,89.13,120473.0,18469.0,26552.0,0.95,18.11,22.04,44.14,125.0,0.70,0.90,37.29,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,58.15,104469.0,26107.0,7919.0,-0.02,33.32,7.58,43.43,77.0,3.30,0.78,39.73,XY24,NTT,MISC
84,UNIONBANK,123.87,163.00,1.23,M-LC,91.30,177304.0,36464.0,8032.0,1.74,25.89,4.53,31.59,88.0,4.54,1.32,54.01,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-24.57,M-MC,75.00,198956.0,19148.0,93271.0,4.00,10.65,46.88,62.52,168.0,0.21,1.49,43.18,XR,NTT,BANKS
90,WHIRLPOOL,1167.49,2270.00,-31.89,M-SC,52.17,198744.0,27123.0,134947.0,4.94,15.80,67.90,94.43,191.0,0.20,1.48,46.69,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-9.71,M-LC,23.37,155006.0,4061.0,105389.0,0.57,2.69,67.99,72.51,70.0,0.04,1.16,7.30,XR,NTT,IT
40,INDIAMART,2327.09,4871.06,-50.72,H-SC,46.74,128128.0,4792.0,130037.0,-0.44,3.89,101.49,109.32,122.0,0.04,0.96,26.58,AR,ATH,MISC
36,HINDZINC,514.80,744.74,39.88,M-LC,93.48,104510.0,1550.0,44438.0,4.04,1.51,42.52,44.67,68.0,0.03,0.78,34.87,X5K,ATH,METALS


In [17]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,54.89,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,260.0,-0.56,0.47,55.40,XR,NTT,HOTELS
5,ALKYLAMINE,2347.98,4567.52,-7.21,X-SC,88.04,86447.0,-14516.0,109952.0,1.45,-14.38,127.19,94.53,59.0,-0.13,0.65,28.17,SR,ATH,CHEMICALS
53,MASFIN,326.60,399.50,-20.05,H-SC,73.91,90900.0,-7080.0,28952.0,2.80,-7.23,31.85,22.32,138.0,-0.24,0.68,31.84,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,58.15,104469.0,26107.0,7919.0,-0.02,33.32,7.58,43.43,77.0,3.30,0.78,39.73,XY24,NTT,MISC
36,HINDZINC,514.80,744.74,39.88,M-LC,93.48,104510.0,1550.0,44438.0,4.04,1.51,42.52,44.67,68.0,0.03,0.78,34.87,X5K,ATH,METALS


In [18]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VALIANTORG,512.64,838.00,-368.25,H-SC,100.00,114101.0,-19185.0,103775.0,1.66,-14.39,90.95,63.47,144.0,-0.18,0.85,91.26,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,6605.56,M-SC,98.91,73595.0,-20215.0,96836.0,0.25,-21.55,131.58,81.67,235.0,-0.21,0.55,42.52,XR,NTT,CERAMICS
80,TITAGARH,1097.23,1548.00,-4.11,H-SC,97.83,168438.0,-23577.0,102461.0,2.30,-12.28,60.83,41.08,152.0,-0.23,1.26,42.56,XY24,NTT,ENGINEERING
76,TANLA,917.30,1963.11,-32.99,H-SC,96.74,164078.0,-75337.0,348288.0,0.93,-31.47,212.27,114.01,127.0,-0.22,1.22,50.48,AR,ATH,IT
47,JPPOWER,18.73,26.20,-34.40,L-SC,95.65,133512.0,-8330.0,64900.0,8.56,-5.87,48.61,39.88,256.0,-0.13,1.00,35.30,XY24,NTT,POWER


In [19]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.38
1,25,43.25
2,50,71.97


In [20]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.51
XY25     15.17
XR       11.28
X40      10.73
X40N      8.36
OX40N     8.24
AR        7.85
X5K       2.30
X200      1.42
SR        1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    18.44
X-LC    17.86
M-SC    16.42
M-LC     9.96
H-MC     8.83
H-LC     7.76
X-MC     6.14
M-MC     5.86
L-SC     3.12
X-SC     2.75
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [22]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.97,-1.02,34.64
IT,10.90,-13.85,71.65
BANKS,10.33,-4.95,51.98
MISC,6.39,-26.38,75.80
FINANCE,6.35,-18.18,63.66
PAINTS,5.56,-17.24,40.35
INSURANCE,5.22,4.73,23.74
ELECTRICAL,5.11,-2.81,47.41
HEALTHCARE,4.65,-3.40,33.60


In [23]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2838102.0,26
XR,1203766.0,14
AR,1018120.0,9
X40,612405.0,8
XY25,495782.0,10
X40N,409461.0,8
OX40N,328261.0,11
SR,189601.0,2
X5K,93267.0,3


In [24]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1897383.0,18
M-SC,1675850.0,20
X-LC,820401.0,11
M-MC,560254.0,5
H-MC,543423.0,7
M-LC,381023.0,8
X-MC,322503.0,5
L-SC,309733.0,5
X-SC,266051.0,4


In [25]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,883286.0,6
M-SC,XY24,728496.0,6
H-SC,AR,537555.0,3
X-LC,X40,465793.0,4
M-SC,XR,421269.0,5
M-MC,XY24,382231.0,3
H-MC,XY24,223395.0,2
H-SC,XR,206073.0,3
M-LC,XY24,202385.0,4


In [26]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
